### Reading the file

In [1]:
import os
import re
import polars as pl
from pathlib import Path


root_dir = os.getcwd()
data_folder = os.path.join(root_dir, 'data')
output_folder = os.path.join(data_folder, 'output')

anime_df = pl.concat([pl.read_ipc(file) for file in Path(output_folder).glob("*.feather")], how='diagonal_relaxed')


In [2]:
anime_df[0]

rank,name,link,score,Type,Episodes,Status,Aired,Premiered,Broadcast,Producers,Licensors,Studios,Source,Genres,Demographic,Duration,Rating,Score,Ranked,Popularity,Members,Favorites,Theme,Themes,Genre,Demographics
i64,str,str,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
1,"""Sousou no Frieren""","""https://myanimelist.net/anime/…",9.31,"""TV""","""28""","""Finished Airing""","""Sep 29, 2023 to Mar 22, 2024""","""Fall 2023""","""Fridays at 23""","""Aniplex, Dentsu, Shogakukan-Sh…","""Crunchyroll""","""Madhouse""","""Manga""","""Adventure, Drama, Fantasy""","""Shounen""","""24 min. per ep.""","""PG-13 - Teens 13 or older""","""9.311 (scored by 615,448 users…","""#12""","""#159""","""1,052,912""","""64,497""",null,null,null,null


### Cleaning the raw data

In [6]:
# Function to clean genres and themes
def clean_col(col1, col2):
    if col1 is None and col2 is None:
        return []
    elif col1 and not col2:
        return [col1]
    elif not col1 and col2:
        return [i.strip() for i in col2.split(",")]
    return [col1] + [i.strip() for i in col2.split(",")]


# Function to clean scores
def clean_scores(score):
    score = str(score).strip()
    
    if not score or score.lower() == "nan":
        return None, None

    match = re.match(r"(\d+\.\d+)\s*\(scored by ([\d,]+) users\)", score)
    
    if match:
        return float(match.group(1)), int(match.group(2).replace(",", ""))
    
    return None, None


def split_and_trim(tags: str | None) -> list[str] | None:
    if not tags:
        return []  # Handle None values
    return [tag.strip() for tag in tags.split(",")]

# Ensure columns are treated as strings
cleaned_anime_df = anime_df\
    .with_columns(
        pl.col("Genre").cast(pl.Utf8, strict=False),
        pl.col("Genres").cast(pl.Utf8, strict=False),
        pl.col("Theme").cast(pl.Utf8, strict=False),
        pl.col("Themes").cast(pl.Utf8, strict=False),
        pl.col("Demographic").cast(pl.Utf8, strict=False),
        pl.col("Demographics").cast(pl.Utf8, strict=False),
        pl.col("Score").cast(pl.Utf8, strict=False)
    )\
    .with_columns(
        # Apply the functions using `.map_elements()`
        pl.struct(["Genre", "Genres"]).map_elements(function=lambda x: clean_col(x["Genre"], x["Genres"]), return_dtype=pl.List(pl.Utf8)).alias("genres_combine"),
        pl.struct(["Theme", "Themes"]).map_elements(function=lambda x: clean_col(x["Theme"], x["Themes"]), return_dtype=pl.List(pl.Utf8)).alias("themes_combine"),
        pl.struct(["Demographic", "Demographics"]).map_elements(function=lambda x: clean_col(x["Demographic"], x["Demographics"]), return_dtype=pl.List(pl.Utf8)).alias("demographics_combine"),
        pl.col("Score").map_elements(
            function=clean_scores,
            return_dtype=pl.List(pl.Float64)
        ).alias("score_data")
    )\
    .with_columns(
        pl.col("score_data").list.get(0).alias("score_cleaned"),
        pl.col("score_data").list.get(1).cast(pl.Int64).alias(name="score_no_of_users")
    )\
    .drop(["Themes", "Theme", "Genres", "Genre", "Demographic", "Demographics", "Score", "score_data"])\
    .with_columns(
        [
        pl.col(col).replace({"None found, add some": ""}) for col in ["Licensors", "Producers", "Studios"]
        ],
    )\
    .with_columns(
        [
        pl.col(col).fill_null("") for col in ["Licensors", "Producers", "Studios"]
        ],
    )\
    .select(
        pl.col("link").str.extract(r"anime/(\d+)", 1).cast(pl.Int64).alias("anime_id"),
        "name",
        "rank",
        "link",
        pl.col("score_cleaned").alias("score"),
        pl.col("score_no_of_users").alias("scored_by_no_of_users"),
        pl.col("Type").alias("type"),
        pl.when(pl.col("Episodes").str.contains("Unknown") | pl.col("Episodes").is_null())
            .then(None)
            .otherwise(pl.col("Episodes")).cast(pl.Int64)
            .alias("episodes"),
        pl.col("Status").alias("status"),
        pl.col("Aired").str.extract(r"(\w{3} \d{1,2}, \d{4})").alias("start_date"),
        pl.col("Aired").str.extract(r"(\w{3} \d{1,2}, \d{4})$", 1).alias("end_date"),
        pl.col("Premiered").alias("premiered"),
        pl.col("Broadcast").alias("broadcast"),
        pl.col("Producers").map_elements(split_and_trim, return_dtype=pl.List(pl.Utf8)).alias("producers"),
        pl.col("Licensors").map_elements(split_and_trim, return_dtype=pl.List(pl.Utf8)).alias("licensors"),
        pl.col("Studios").map_elements(split_and_trim, return_dtype=pl.List(pl.Utf8)).alias("studios"),
        pl.col("Source").alias("source"),
        pl.col("Duration").alias("duration"),
        pl.col("Rating").alias("rating"),
        pl.col("Ranked").str.extract(r"(\d+)").cast(pl.Int64).alias("ranked"),
        pl.col("Popularity").str.extract(r"(\d+)").cast(pl.Int64).alias("popularity"),
        pl.col("Members").str.replace_all(",", "").cast(pl.Int64).alias("members"),
        pl.col("Favorites").str.replace_all(",", "").cast(pl.Int64).alias("favorites"),
        pl.col("genres_combine").alias("genres"),
        pl.col("themes_combine").alias("themes"),
        pl.col("demographics_combine").alias("demographics"),
    )\
    .with_columns(
        pl.col("start_date").str.strptime(pl.Date, "%b %d, %Y"),
        pl.col("end_date").str.strptime(pl.Date, "%b %d, %Y"),
    )\
    .with_columns(
        pl.when(pl.col("start_date") == pl.col("end_date"))
            .then(None)  # Set to null when start_date and end_date are the same
            .otherwise((pl.col("end_date") - pl.col("start_date")))
            .alias("length_days")
    )\
    .with_columns(
        pl.col("length_days").dt.total_days().cast(pl.Int64).alias("length_days"),
    )


In [7]:
cleaned_anime_df.write_ipc(os.path.join(data_folder, "cleaned", "anime_cleaned.feather"))
cleaned_anime_df.write_parquet(os.path.join(data_folder, "cleaned", "anime_cleaned.parquet"))

In [4]:
pl.read_parquet(os.path.join(data_folder, "cleaned", "anime_cleaned.parquet"))[0]

anime_id,name,rank,link,score,scored_by_no_of_users,type,episodes,status,start_date,end_date,premiered,broadcast,producers,licensors,studios,source,duration,rating,ranked,popularity,members,favorites,genres,themes,demographics,length_days
i64,str,i64,str,f64,i64,str,i64,str,date,date,str,str,list[str],list[str],list[str],str,str,str,i64,i64,i64,i64,list[str],list[str],list[str],i64
52991,"""Sousou no Frieren""",1,"""https://myanimelist.net/anime/…",9.311,615448,"""TV""",28,"""Finished Airing""",2023-09-29,2024-03-22,"""Fall 2023""","""Fridays at 23""","[""Aniplex"", ""Dentsu"", … ""Shogakukan""]","[""Crunchyroll""]","[""Madhouse""]","""Manga""","""24 min. per ep.""","""PG-13 - Teens 13 or older""",12,159,1052912,64497,"[""Adventure"", ""Drama"", ""Fantasy""]",[],"[""Shounen""]",175


### Prepare dataset for neo4j

In [9]:
cleaned_df = pl.read_parquet(os.path.join(data_folder, "cleaned", "anime_cleaned.parquet"))

cleaned_df[0]

anime_id,name,rank,link,score,scored_by_no_of_users,type,episodes,status,start_date,end_date,premiered,broadcast,producers,licensors,studios,source,duration,rating,ranked,popularity,members,favorites,genres,themes,demographics,length_days
i64,str,i64,str,f64,i64,str,i64,str,date,date,str,str,list[str],list[str],list[str],str,str,str,i64,i64,i64,i64,list[str],list[str],list[str],i64
52991,"""Sousou no Frieren""",1,"""https://myanimelist.net/anime/…",9.311,615448,"""TV""",28,"""Finished Airing""",2023-09-29,2024-03-22,"""Fall 2023""","""Fridays at 23""","[""Aniplex"", ""Dentsu"", … ""Shogakukan""]","[""Crunchyroll""]","[""Madhouse""]","""Manga""","""24 min. per ep.""","""PG-13 - Teens 13 or older""",12,159,1052912,64497,"[""Adventure"", ""Drama"", ""Fantasy""]",[],"[""Shounen""]",175


In [10]:
# Prepare cleaned data for Neo4j import by creating nodes and relationships

# 1. Create node DataFrames with unique IDs
# Main anime nodes
main_anime_df = cleaned_df.select([
    "anime_id",  # Using existing anime_id as unique identifier
    "name",
    "link", 
    "score",
    "scored_by_no_of_users",
    "episodes",
    "start_date",
    "end_date",
    "premiered",
    "broadcast", 
    "duration",
    "ranked",
    "popularity",
    "members", 
    "favorites",
    "length_days"
])

# Create nodes for categorical attributes with unique IDs
def create_node_df(col_name):
    unique_values = cleaned_df.select(pl.col(col_name).unique()).to_series().to_list()
    return pl.DataFrame({
        f"{col_name}_id": range(1, len(unique_values) + 1),
        col_name: unique_values
    })

# Create node DataFrames for non-list columns
categorical_nodes = {
    "source": create_node_df("source"),
    "type": create_node_df("type"), 
    "rating": create_node_df("rating"),
    "status": create_node_df("status")
}

# Create node DataFrames for list columns
list_columns = ["genres", "themes", "demographics", "producers", "licensors", "studios"]
for col in list_columns:
    unique_values = cleaned_df.explode(col).select(pl.col(col).unique()).to_series().to_list()
    categorical_nodes[col] = pl.DataFrame({
        f"{col[:-1]}_id": range(1, len(unique_values) + 1),  # Remove 's' from column name
        col[:-1]: unique_values  # Remove 's' from column name
    })

# 2. Create relationship DataFrames
# For non-list columns
relationship_dfs = {}
for col in ["source", "type", "rating", "status"]:
    relationship_dfs[f"anime_{col}"] = (
        cleaned_df
        .join(categorical_nodes[col], left_on=col, right_on=col)
        .select(["anime_id", f"{col}_id"])
    )

# For list columns 
for col in list_columns:
    singular = col[:-1]  # Remove 's' from column name
    relationship_dfs[f"anime_{col}"] = (
        cleaned_df
        .explode(col)
        .join(categorical_nodes[col], left_on=col, right_on=singular)
        .select(["anime_id", f"{singular}_id"])
    )

# 3. Save all DataFrames to CSV
neo4j_folder = os.path.join(data_folder, "neo4j")
os.makedirs(neo4j_folder, exist_ok=True)

# Save main anime nodes
main_anime_df.write_csv(os.path.join(neo4j_folder, "anime_nodes.csv"))

# Save categorical nodes
for name, df in categorical_nodes.items():
    df.write_csv(os.path.join(neo4j_folder, f"{name}_nodes.csv"))

# Save relationships
for name, df in relationship_dfs.items():
    df.write_csv(os.path.join(neo4j_folder, f"{name}_rels.csv"))

In [ ]:
# Neo4j Cypher queries for loading the data


# Node creation queries
node_queries = [
    # Main anime nodes
    """
    LOAD CSV WITH HEADERS FROM 'file:///anime_nodes.csv' AS row
    CREATE (:Anime {
        anime_id: toInteger(row.anime_id),
        name: row.name,
        rank: toInteger(row.rank),
        score: toFloat(row.score),
        scored_by: toInteger(row.scored_by),
        popularity: toInteger(row.popularity),
        members: toInteger(row.members),
        favorites: toInteger(row.favorites),
        episodes: toInteger(row.episodes),
        aired_from: row.aired_from,
        aired_to: row.aired_to,
        length_days: toInteger(row.length_days)
    })
    """,
    
    # Categorical nodes
    """
    LOAD CSV WITH HEADERS FROM 'file:///source_nodes.csv' AS row
    CREATE (:Source {source_id: toInteger(row.source_id), source: row.source})
    """,
    
    """
    LOAD CSV WITH HEADERS FROM 'file:///type_nodes.csv' AS row
    CREATE (:Type {type_id: toInteger(row.type_id), type: row.type})
    """,
    
    """
    LOAD CSV WITH HEADERS FROM 'file:///rating_nodes.csv' AS row
    CREATE (:Rating {rating_id: toInteger(row.rating_id), rating: row.rating})
    """,
    
    """
    LOAD CSV WITH HEADERS FROM 'file:///status_nodes.csv' AS row
    CREATE (:Status {status_id: toInteger(row.status_id), status: row.status})
    """,
    
    """
    LOAD CSV WITH HEADERS FROM 'file:///genre_nodes.csv' AS row
    CREATE (:Genre {genre_id: toInteger(row.genre_id), genre: row.genre})
    """,
    
    """
    LOAD CSV WITH HEADERS FROM 'file:///theme_nodes.csv' AS row
    CREATE (:Theme {theme_id: toInteger(row.theme_id), theme: row.theme})
    """,
    
    """
    LOAD CSV WITH HEADERS FROM 'file:///demographic_nodes.csv' AS row
    CREATE (:Demographic {demographic_id: toInteger(row.demographic_id), demographic: row.demographic})
    """,
    
    """
    LOAD CSV WITH HEADERS FROM 'file:///producer_nodes.csv' AS row
    CREATE (:Producer {producer_id: toInteger(row.producer_id), producer: row.producer})
    """,
    
    """
    LOAD CSV WITH HEADERS FROM 'file:///licensor_nodes.csv' AS row
    CREATE (:Licensor {licensor_id: toInteger(row.licensor_id), licensor: row.licensor})
    """,
    
    """
    LOAD CSV WITH HEADERS FROM 'file:///studio_nodes.csv' AS row
    CREATE (:Studio {studio_id: toInteger(row.studio_id), studio: row.studio})
    """
]

# Relationship creation queries
relationship_queries = [
    # Non-list relationships
    """
    LOAD CSV WITH HEADERS FROM 'file:///anime_source_rels.csv' AS row
    MATCH (a:Anime {anime_id: toInteger(row.anime_id)})
    MATCH (s:Source {source_id: toInteger(row.source_id)})
    CREATE (a)-[:HAS_SOURCE]->(s)
    """,
    
    """
    LOAD CSV WITH HEADERS FROM 'file:///anime_type_rels.csv' AS row
    MATCH (a:Anime {anime_id: toInteger(row.anime_id)})
    MATCH (t:Type {type_id: toInteger(row.type_id)})
    CREATE (a)-[:HAS_TYPE]->(t)
    """,
    
    """
    LOAD CSV WITH HEADERS FROM 'file:///anime_rating_rels.csv' AS row
    MATCH (a:Anime {anime_id: toInteger(row.anime_id)})
    MATCH (r:Rating {rating_id: toInteger(row.rating_id)})
    CREATE (a)-[:HAS_RATING]->(r)
    """,
    
    """
    LOAD CSV WITH HEADERS FROM 'file:///anime_status_rels.csv' AS row
    MATCH (a:Anime {anime_id: toInteger(row.anime_id)})
    MATCH (s:Status {status_id: toInteger(row.status_id)})
    CREATE (a)-[:HAS_STATUS]->(s)
    """,
    
    # List relationships
    """
    LOAD CSV WITH HEADERS FROM 'file:///anime_genres_rels.csv' AS row
    MATCH (a:Anime {anime_id: toInteger(row.anime_id)})
    MATCH (g:Genre {genre_id: toInteger(row.genre_id)})
    CREATE (a)-[:HAS_GENRE]->(g)
    """,
    
    """
    LOAD CSV WITH HEADERS FROM 'file:///anime_themes_rels.csv' AS row
    MATCH (a:Anime {anime_id: toInteger(row.anime_id)})
    MATCH (t:Theme {theme_id: toInteger(row.theme_id)})
    CREATE (a)-[:HAS_THEME]->(t)
    """,
    
    """
    LOAD CSV WITH HEADERS FROM 'file:///anime_demographics_rels.csv' AS row
    MATCH (a:Anime {anime_id: toInteger(row.anime_id)})
    MATCH (d:Demographic {demographic_id: toInteger(row.demographic_id)})
    CREATE (a)-[:HAS_DEMOGRAPHIC]->(d)
    """,
    
    """
    LOAD CSV WITH HEADERS FROM 'file:///anime_producers_rels.csv' AS row
    MATCH (a:Anime {anime_id: toInteger(row.anime_id)})
    MATCH (p:Producer {producer_id: toInteger(row.producer_id)})
    CREATE (a)-[:PRODUCED_BY]->(p)
    """,
    
    """
    LOAD CSV WITH HEADERS FROM 'file:///anime_licensors_rels.csv' AS row
    MATCH (a:Anime {anime_id: toInteger(row.anime_id)})
    MATCH (l:Licensor {licensor_id: toInteger(row.licensor_id)})
    CREATE (a)-[:LICENSED_BY]->(l)
    """,
    
    """
    LOAD CSV WITH HEADERS FROM 'file:///anime_studios_rels.csv' AS row
    MATCH (a:Anime {anime_id: toInteger(row.anime_id)})
    MATCH (s:Studio {studio_id: toInteger(row.studio_id)})
    CREATE (a)-[:PRODUCED_BY_STUDIO]->(s)
    """
]

# Print all queries for copying into Neo4j browser
print("// Create indexes")
print("CREATE INDEX anime_id_index FOR (n:Anime) ON (n.anime_id);")
print()

print("// Node creation queries")
for query in node_queries:
    print(query)
    print()

print("// Relationship creation queries")
for query in relationship_queries:
    print(query)
    print()
